In [ ]:
from __future__ import division

from models import *
from utils.logger import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *
from test import evaluate

from terminaltables import AsciiTable

import os
import sys
import time
import datetime
import argparse
import easydict

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    args = parser.parse_args(args=[])
#     parser.add_argument("--epochs", type=int, default=100, help="number of epochs")
#     parser.add_argument("--batch_size", type=int, default=8, help="size of each image batch")
#     parser.add_argument("--gradient_accumulations", type=int, default=2, help="number of gradient accums before step")
#     parser.add_argument("--model_def", type=str, default="config/yolov3.cfg", help="path to model definition file")
#     parser.add_argument("--data_config", type=str, default="config/coco.data", help="path to data config file")
#     parser.add_argument("--pretrained_weights", type=str, help="if specified starts from checkpoint model")
#     parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
#     parser.add_argument("--img_size", type=int, default=416, help="size of each image dimension")
#     parser.add_argument("--checkpoint_interval", type=int, default=1, help="interval between saving model weights")
#     parser.add_argument("--evaluation_interval", type=int, default=1, help="interval evaluations on validation set")
#     parser.add_argument("--compute_map", default=False, help="if True computes mAP every tenth batch")
#     parser.add_argument("--multiscale_training", default=True, help="allow for multi-scale training")
#     opt = parser.parse_args()
    args = easydict.EasyDict({
        "epochs": 160,
        "batch_size": 4,
        "gradient_accumulations": 2,
        "model_def": r'D:\Ivan\Test_data\IvanMadeDataSet\Yolo_front\config/yolov3.cfg',
        "data_config": r'D:\Ivan\Test_data\IvanMadeDataSet\Yolo_front\config/coco.data',
        "pretrained_weights": '',
        "n_cpu": 2,
        "img_size": 416,
        "checkpoint_interval": 1,
        "evaluation_interval": 1,
        "compute_map": False,
        "multiscale_training": True
    })
    opt = args
    print(opt)
    
    outputDir = r"D:\Ivan\YoloCheckpoints/OID_front_1_erkli_car/"
    logger = Logger(outputDir+"logs")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    os.makedirs(outputDir+"output", exist_ok=True)
    os.makedirs(outputDir+"checkpoints", exist_ok=True)

    # Get data configuration
    data_config = parse_data_config(opt.data_config)
    train_path = data_config["train"]
    valid_path = data_config["valid"]
    class_names = load_classes(data_config["names"])

    # Initiate model
    model = Darknet(opt.model_def).to(device)
    model.apply(weights_init_normal)

    # If specified we start from checkpoint
    if opt.pretrained_weights:
        if opt.pretrained_weights.endswith(".pth"):
            model.load_state_dict(torch.load(opt.pretrained_weights))
        else:
            model.load_darknet_weights(opt.pretrained_weights)

    # Get dataloader
    dataset = ListDataset(train_path, augment=True, multiscale=opt.multiscale_training)
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=opt.batch_size,
        shuffle=True,
        num_workers=opt.n_cpu,
        pin_memory=True,
        collate_fn=dataset.collate_fn,
    )

    optimizer = torch.optim.Adam(model.parameters())

    metrics = [
        "grid_size",
        "loss",
        "x",
        "y",
        "w",
        "h",
        "conf",
        "cls",
        "cls_acc",
        "recall50",
        "recall75",
        "precision",
        "conf_obj",
        "conf_noobj",
    ]

    for epoch in range(opt.epochs):
        model.train()
        start_time = time.time()
        for batch_i, (_, imgs, targets) in enumerate(dataloader):
            batches_done = len(dataloader) * epoch + batch_i

            imgs = Variable(imgs.to(device))
            targets = Variable(targets.to(device), requires_grad=False)

            loss, outputs = model(imgs, targets)
            loss.backward()

            if batches_done % opt.gradient_accumulations:
                # Accumulates gradient before each step
                optimizer.step()
                optimizer.zero_grad()

            # ----------------
            #   Log progress
            # ----------------

            log_str = "\n---- [Epoch %d/%d, Batch %d/%d] ----\n" % (epoch, opt.epochs, batch_i, len(dataloader))

            metric_table = [["Metrics", *[f"YOLO Layer {i}" for i in range(len(model.yolo_layers))]]]

            # Log metrics at each YOLO layer
            for i, metric in enumerate(metrics):
                formats = {m: "%.6f" for m in metrics}
                formats["grid_size"] = "%2d"
                formats["cls_acc"] = "%.2f%%"
                row_metrics = [formats[metric] % yolo.metrics.get(metric, 0) for yolo in model.yolo_layers]
                metric_table += [[metric, *row_metrics]]

                # Tensorboard logging
                tensorboard_log = []
                for j, yolo in enumerate(model.yolo_layers):
                    for name, metric in yolo.metrics.items():
                        if name != "grid_size":
                            tensorboard_log += [(f"{name}_{j+1}", metric)]
                tensorboard_log += [("loss", loss.item())]
#                 logger.list_of_scalars_summary(tensorboard_log, batches_done)

            log_str += AsciiTable(metric_table).table
            log_str += f"\nTotal loss {loss.item()}"

            # Determine approximate time left for epoch
            epoch_batches_left = len(dataloader) - (batch_i + 1)
            time_left = datetime.timedelta(seconds=epoch_batches_left * (time.time() - start_time) / (batch_i + 1))
            log_str += f"\n---- ETA {time_left}"

            print(log_str)

            model.seen += imgs.size(0)

        if epoch % opt.evaluation_interval == 0:
            print("\n---- Evaluating Model ----")
            # Evaluate the model on the validation set
            precision, recall, AP, f1, ap_class = evaluate(
                model,
                path=valid_path,
                iou_thres=0.5,
                conf_thres=0.5,
                nms_thres=0.5,
                img_size=opt.img_size,
                batch_size=8,
            )
#             evaluation_metrics = [
#                 ("val_precision", precision.mean()),
#                 ("val_recall", recall.mean()),
#                 ("val_mAP", AP.mean()),
#                 ("val_f1", f1.mean()),
#             ]
            logger.scalar_summary("val_precision", precision.mean(),epoch)
            logger.scalar_summary("val_recall", recall.mean(), epoch)
            logger.scalar_summary("val_mAp", AP.mean())
            logger.scalar_summary("val_f1", f1.mean())
#             logger.list_of_scalars_summary(evaluation_metrics, epoch)

            # Print class APs and mAP
            ap_table = [["Index", "Class name", "AP"]]
            for i, c in enumerate(ap_class):
                ap_table += [[c, class_names[c], "%.5f" % AP[i]]]
            print(AsciiTable(ap_table).table)
            print(f"---- mAP {AP.mean()}")

        if epoch % opt.checkpoint_interval == 0:
            torch.save(model.state_dict(), outputDir+f"checkpoints/yolov3_ckpt_%d.pth" % epoch)


{'epochs': 160, 'batch_size': 4, 'gradient_accumulations': 2, 'model_def': 'D:\\Ivan\\Test_data\\IvanMadeDataSet\\Yolo_front\\config/yolov3.cfg', 'data_config': 'D:\\Ivan\\Test_data\\IvanMadeDataSet\\Yolo_front\\config/coco.data', 'pretrained_weights': '', 'n_cpu': 2, 'img_size': 416, 'checkpoint_interval': 1, 'evaluation_interval': 1, 'compute_map': False, 'multiscale_training': True}


C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.ui


---- [Epoch 0/160, Batch 0/1757] ----
+------------+--------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 | YOLO Layer 2 |
+------------+--------------+--------------+--------------+
| grid_size  | 16           | 32           | 64           |
| loss       | 66.795052    | 74.793129    | 80.128410    |
| x          | 0.093217     | 0.082323     | 0.081983     |
| y          | 0.191933     | 0.146708     | 0.092655     |
| w          | 0.514355     | 1.441955     | 4.170001     |
| h          | 0.831924     | 0.658829     | 3.959963     |
| conf       | 64.384621    | 71.554161    | 71.013077    |
| cls        | 0.778997     | 0.909157     | 0.810729     |
| cls_acc    | 100.00%      | 100.00%      | 100.00%      |
| recall50   | 0.200000     | 0.200000     | 0.200000     |
| recall75   | 0.000000     | 0.000000     | 0.000000     |
| precision  | 0.001033     | 0.000157     | 0.000037     |
| conf_obj   | 0.436479     | 0.570485     | 0.602642     |
|

C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.ui


---- [Epoch 0/160, Batch 1/1757] ----
+------------+--------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 | YOLO Layer 2 |
+------------+--------------+--------------+--------------+
| grid_size  | 12           | 24           | 48           |
| loss       | 66.766663    | 73.334122    | 74.677658    |
| x          | 0.072540     | 0.124753     | 0.114734     |
| y          | 0.149116     | 0.125652     | 0.142537     |
| w          | 0.789292     | 0.608307     | 1.381877     |
| h          | 0.455206     | 0.106145     | 1.032753     |
| conf       | 64.456779    | 71.784988    | 71.160484    |
| cls        | 0.843726     | 0.584274     | 0.845272     |
| cls_acc    | 100.00%      | 100.00%      | 100.00%      |
| recall50   | 0.000000     | 0.200000     | 0.000000     |
| recall75   | 0.000000     | 0.000000     | 0.000000     |
| precision  | 0.000000     | 0.000281     | 0.000000     |
| conf_obj   | 0.407730     | 0.485116     | 0.519480     |
|

...

C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.ui


---- [Epoch 0/160, Batch 23/1757] ----
+------------+--------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 | YOLO Layer 2 |
+------------+--------------+--------------+--------------+
| grid_size  | 15           | 30           | 60           |
| loss       | 8.761649     | 15.669684    | 36.958618    |
| x          | 0.066202     | 0.033882     | 0.129143     |
| y          | 0.144014     | 0.157681     | 0.095943     |
| w          | 0.104230     | 1.139757     | 3.583769     |
| h          | 0.231986     | 0.206599     | 3.673913     |
| conf       | 7.920578     | 13.946278    | 29.052986    |
| cls        | 0.294640     | 0.185489     | 0.422862     |
| cls_acc    | 100.00%      | 100.00%      | 100.00%      |
| recall50   | 0.000000     | 0.000000     | 0.000000     |
| recall75   | 0.000000     | 0.000000     | 0.000000     |
| precision  | 0.000000     | 0.000000     | 0.000000     |
| conf_obj   | 0.048070     | 0.136491     | 0.267088     |


C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.ui


---- [Epoch 0/160, Batch 24/1757] ----
+------------+--------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 | YOLO Layer 2 |
+------------+--------------+--------------+--------------+
| grid_size  | 12           | 24           | 48           |
| loss       | 8.652321     | 15.281868    | 30.301790    |
| x          | 0.159957     | 0.068393     | 0.103361     |
| y          | 0.197019     | 0.183907     | 0.084985     |
| w          | 0.335517     | 0.921730     | 1.102321     |
| h          | 0.740930     | 0.500559     | 1.352706     |
| conf       | 7.070369     | 13.368898    | 27.384233    |
| cls        | 0.148529     | 0.238381     | 0.274184     |
| cls_acc    | 100.00%      | 100.00%      | 100.00%      |
| recall50   | 0.000000     | 0.000000     | 0.000000     |
| recall75   | 0.000000     | 0.000000     | 0.000000     |
| precision  | 0.000000     | 0.000000     | 0.000000     |
| conf_obj   | 0.052342     | 0.145019     | 0.277277     |


C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\cb\pytorch_1000000000000\work\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.ui